In [ ]:
import tensorflow as tf

imageHeight = 512
imageWidth = 512

filterDim = [256, 384, 384, 384, 512]

In [ ]:
l = tf.keras.layers
def _conv(inputs, filters, kernel_size, strides, padding, bias=False, normalize=True, activation='relu'):
    output = inputs
    padding_str = 'same'
    if padding>0:
        output = l.ZeroPadding2D(padding=padding)(output)
        padding_str = 'valid'
    output = l.Conv2D(filters, kernel_size, strides, padding_str, use_bias=bias, \
                 kernel_initializer='he_normal', \
                 kernel_regularizer=tf.keras.regularizers.l2(l=5e-4))(output)
    if normalize:
        output = l.BatchNormalization(axis=3)(output)
    if activation=='relu':
        output = l.ReLU()(output)
    if activation=='relu6':
        output = l.ReLU(max_value=6)(output)
    if activation=='leaky_relu':
        output = l.LeakyReLU(alpha=0.1)(output)
    return output

def _residual(inputs, filters, strides):
    shortcut = inputs
    num_channels = shortcut.get_shape().as_list()[-1]
    output = _conv(inputs, filters, 3, 2, 1)
    output = _conv(output, filters, 3, 1, 0, False, True, 'linear')
    if num_channels != filters or strides != 1:
        shortcut = _conv(shortcut, filters, 1, strides, 0, False, Ture, 'linear')
    output = l.Add()([output, shortcut])
    output = l.ReLU()(output)
    return output

def _hourglass(inputs, filterDim):
    #Left part, left_features image dimension, 1,1/2,1/4,1/8,1/16,1/32
    left_features = [inputs]
    for index, dim in enumerate(filterDim):
        output = _residual(left_features[-1], dim, 2)
        output = _residual(output, dim, 1)
        left_features.append(output)
    #Middle part
    output = left_features[-1]
    for i in range(5):
        output = _residual(output, filterDim[-1], 1)
    #Right part
    for index in reversed(range(len(filterDim))):
        output = _residual(output, filterDim[index], 1)
        output = _residual(output, filterDim[max(index-1, 0)], 1)
        output = l.UpSampling2D()(output)
        left_feature = _residual(left_features[index], filterDim[max(index-1, 0)], 1)
        left_feature = _residual(left_feature, filterDim[max(index-1, 0)], 1)
        output = l.Add()([output, left_feature])
    output = _conv(output, 256, 3, 1, 1)
    #Prediction
    prediction = [output]
    pred_classes = _conv(output, 256, 3, 1, 0, True, False, 'relu') 
    pred_classes = _conv(pred_classes, 80, 1, 1, 0, True, False, 'linear') 
    prediction.append(pred_classes)
    pred_center = _conv(output, 256, 3, 1, 0, True, False, 'relu') 
    pred_center = _conv(pred_center, 2, 1, 1, 0, True, False, 'linear') 
    prediction.append(pred_center)
    pred_wh = _conv(output, 256, 3, 1, 0, True, False, 'relu') 
    pred_wh = _conv(pred_wh, 2, 1, 1, 0, True, False, 'linear') 
    prediction.append(pred_wh)
    return prediction

def HourglassNetwork():
    image = tf.keras.Input(shape=(imageHeight,imageWidth,3))   #512×512×3
    net = _conv(image, 128, 7, 2, 3)    #256×256×128
    net = _residual(net, 256, 2)        #128×128×256
    net, pred1 = _hourglass(net, filterDim)
    net, pred2 = _hourglass(net, filterDim)
    

In [1]:
pow_str = 'center.' * 5

In [2]:
pow_str

'center.center.center.center.center.'

In [3]:
dims = [256, 384, 384, 384, 512]
for kk in reversed(range(len(dims))):
    print(kk)

4
3
2
1
0
